In [1]:
import tensorflow as tf

#retorna o modelo da GPU que está sendo usada:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


## **Dowload do YOLO**

In [4]:
!git clone https://github.com/ultralytics/yolov5.git


Cloning into 'yolov5'...
remote: Enumerating objects: 12270, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 12270 (delta 5), reused 9 (delta 1), pack-reused 12250
Receiving objects: 100% (12270/12270), 11.98 MiB | 8.94 MiB/s, done.
Resolving deltas: 100% (8491/8491), done.


# **Abrir a pasta:**

In [6]:
%cd yolov5

/content/yolov5


# **Instalando os requerimentos:**

In [ ]:
!pip install -r requirements.txt

In [8]:
!ls

CONTRIBUTING.md  export.py   models	       setup.cfg       utils
data		 hubconf.py  README.md	       train.py        val.py
detect.py	 LICENSE     requirements.txt  tutorial.ipynb


# **Rodando o detect.py:**

In [ ]:
!python detect.py 
#python detect.py --source nomedoarquivo


In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images

In [19]:
!display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

/bin/bash: -c: line 0: syntax error near unexpected token `filename='runs/detect/exp/zidane.jpg','
/bin/bash: -c: line 0: `display.Image(filename='runs/detect/exp/zidane.jpg', width=600)'


# **Interface grafica da IA:**

In [26]:
#Dependencias
import cv2
import time

# Cores das classes
COLORS = [(0,255,255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

#Carrega as classes
class_names = []
with open("data/coco.yaml", "r") as f:
  class_names = [cname.strip() for cname in f.readlines()]

cap = cv2.VideoCapture("0") #video da camera

#Carrega os pesos
net = cv2 .dnn.readNet("yolov5.weights", "yolov5.cfg")

#Setando os paramentros da rede neural
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255)

#lendo os frames do video
while True:

  #captura os frames  
  _, frame = cap.read()

  #começo dos ms
  start = time.time()

  #detecção
  classes, scores, boxes = model.detect(frame, 0.05, 0.2)

  #ffim da contagem dos ms
  end = time.time()

  #percorrer todas as detecções 
  for(classid, score, box) in zip(classes, scores, boxes):

    #gerando uma cor para classe
    color = COLORS[int(classid) % len(COLORS)]
    # print(classid)
    #pegando o nome da classe pelo ID e o seu Score 
    label = f"{class_names[classid]} : {score}"

    #desenhando a box da detecção
    cv2.rectangle(frame, box, color, 2)

    #escrevendo o nome da classe em cima da box objeto
    cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  #calculando o tempo que levou para fazer a detecção
  fps_label = f"FPS: {round((1.0/(end - start)), 2)}"

  #Escrevendo o fps na imagem
  cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 5)
  cv2.putText(frame, fps_label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

  #mostando a imagem
  cv2.imshow("dections", frame)

  #espera da resposta
  if cv2.waitKey(1) == 27:
    break

#Liberação da camera e destroi todos as janelas
cap.release()
cv2.destroyAllWindows()

error: ignored

# **Web Can:**

In [29]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))